In [1]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import mlflow

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
X = train.drop(columns=[
    'ID', 'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])

y = train.drop(columns=[
    'ID', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
    'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
    'feature12', 'feature13', 'feature14', 'feature15'
])


In [4]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Hyper-param optmization

In [5]:
import optuna
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score, KFold
import mlflow
import numpy as np
import warnings

warnings.filterwarnings('ignore')

def objective(trial):
    with mlflow.start_run(run_name=f'Trial - {trial.number}', nested=True):
        # Hiperparâmetros do MLP
        params = {
            'hidden_layer_sizes': tuple(
                trial.suggest_int(f'n_units_l{i}', 16, 256, log=True)
                for i in range(trial.suggest_int('n_layers', 1, 3))
            ),
            'activation': trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic']),
            'solver': trial.suggest_categorical('solver', ['adam', 'sgd']),
            'alpha': trial.suggest_float('alpha', 1e-6, 1e-1, log=True),          # L2 penalty
            'learning_rate': trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive']),
            'learning_rate_init': trial.suggest_float('lr_init', 1e-4, 1e-1, log=True),
            'max_iter': trial.suggest_int('max_iter', 200, 5000),
            'early_stopping': True,
            'random_state': 42
        }

        mlflow.log_params(params)

        model = MLPRegressor(**params)

        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        scores = cross_val_score(
            model, X, y,
            scoring='neg_root_mean_squared_error',
            cv=kf
        )

        rmse = -scores.mean()
        mlflow.log_metric("rmse", rmse)
        return rmse

# Para rodar o estudo Optuna:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)


c:\Users\Guilherme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=20,
        n_warmup_steps=10,
        interval_steps=5
    )
)

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('DSWA - Competicao3')
with mlflow.start_run(run_name='MLP optimization'):
    study.optimize(objective, n_trials=50)
    
    print("Melhores hiperparâmetros:")
    print(study.best_params)
    print(f"Melhor RMSE: {study.best_value:.4f}")

    # Loga os melhores parâmetros e métrica no MLflow
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)


[I 2025-07-27 17:23:50,841] A new study created in memory with name: no-name-869436ca-35af-4892-bd4a-54858e148170
[I 2025-07-27 17:24:00,932] Trial 0 finished with value: 0.5360699167712938 and parameters: {'n_layers': 2, 'n_units_l0': 223, 'n_units_l1': 121, 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.0010129197956845735, 'learning_rate': 'adaptive', 'lr_init': 0.03142880890840111, 'max_iter': 1219}. Best is trial 0 with value: 0.5360699167712938.


🏃 View run Trial - 0 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/62820e63ad594a00b96dffa1cef5b2ac
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:24:03,767] Trial 1 finished with value: 0.6131356173823246 and parameters: {'n_layers': 1, 'n_units_l0': 26, 'activation': 'tanh', 'solver': 'sgd', 'alpha': 4.982752357076453e-06, 'learning_rate': 'adaptive', 'lr_init': 0.0226739865237804, 'max_iter': 1158}. Best is trial 0 with value: 0.5360699167712938.


🏃 View run Trial - 1 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/6905fcaf9c5141b3afb5f214a423db3a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:24:08,599] Trial 2 finished with value: 0.774554912476001 and parameters: {'n_layers': 2, 'n_units_l0': 82, 'n_units_l1': 18, 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.01101505679026964, 'learning_rate': 'adaptive', 'lr_init': 0.0020914981329035616, 'max_iter': 785}. Best is trial 0 with value: 0.5360699167712938.


...

🏃 View run Trial - 47 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/58b23f64c4d54dc0bfd5b96be9d38ca4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:34:25,079] Trial 48 finished with value: 0.8063547206294237 and parameters: {'n_layers': 2, 'n_units_l0': 92, 'n_units_l1': 73, 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.004591462029252592, 'learning_rate': 'invscaling', 'lr_init': 0.015513466546379516, 'max_iter': 1064}. Best is trial 21 with value: 0.5294955428402845.


🏃 View run Trial - 48 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/6bf7d0dfa3de4b5998b5b710edafc616
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:34:33,572] Trial 49 finished with value: 0.8499966655848734 and parameters: {'n_layers': 3, 'n_units_l0': 111, 'n_units_l1': 145, 'n_units_l2': 185, 'activation': 'logistic', 'solver': 'adam', 'alpha': 0.01874373507365026, 'learning_rate': 'adaptive', 'lr_init': 0.08052943057242273, 'max_iter': 4986}. Best is trial 21 with value: 0.5294955428402845.


🏃 View run Trial - 49 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/cb766aded04948e7b96356fdf43e0c07
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
Melhores hiperparâmetros:
{'n_layers': 3, 'n_units_l0': 157, 'n_units_l1': 105, 'n_units_l2': 245, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0019676546308791515, 'learning_rate': 'constant', 'lr_init': 0.010895231461708607, 'max_iter': 416}
Melhor RMSE: 0.5295
🏃 View run MLP optimization at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/d68bd399e788412b8d8669496652b0a1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
